## Travel Distance Calculation with Json

In [ ]:
import pandas as pd
import numpy as np
# Import Station Information
station_2019 = pd.read_csv('C:/Users/24707/Desktop/Bike/distance/station2019.csv', delimiter=',')
station_2019 = station_2019['station name'].to_list()
station_2020 = pd.read_csv('C:/Users/24707/Desktop/Bike/distance/station2020.csv', delimiter=',')
station_2020 = station_2020['station name'].to_list()
station_2021 = pd.read_csv('C:/Users/24707/Desktop/Bike/distance/station2021.csv', delimiter=',')
station_2021 = station_2021['station name'].to_list()
months = ['01','02','03','04','05','06','07','08','09','10','11','12']


In [ ]:
# 2019
frame = []
for month in months:
    df =  pd.read_csv("./2019/2019"+month+"-citibike-tripdata.csv", delimiter=',')
    df = df[['tripduration','starttime','start station name','end station name','usertype',"start station latitude","start station longitude","end station latitude","end station longitude"]]
    df = df[(df['start station name'].isin(station_2019))&(df['end station name'].isin(station_2019))]
    #df = df.drop_duplicates(subset=["start station latitude","start station longitude","end station latitude","end station longitude"])
    df = df.drop(labels=['start station name','end station name'], axis=1)
    df = df.dropna(axis=0, how='any', subset=["start station latitude","start station longitude","end station latitude","end station longitude"])
    frame.append(df)
result_2019 = pd.concat(frame)


In [ ]:
#2020
frame = []
for month in months:
    df =  pd.read_csv("./2020/2020"+month+"-citibike-tripdata.csv", delimiter=',')
    df = df[['tripduration','starttime','start station name','end station name','usertype',"start station latitude","start station longitude","end station latitude","end station longitude"]]
    df = df[(df['start station name'].isin(station_2020))&(df['end station name'].isin(station_2020))]
    #df = df.drop_duplicates(subset=["start station latitude","start station longitude","end station latitude","end station longitude"])
    df = df.drop(labels=['start station name','end station name'], axis=1)
    df = df.dropna(axis=0, how='any', subset=["start station latitude","start station longitude","end station latitude","end station longitude"])
    frame.append(df)
result_2020 = pd.concat(frame)


In [ ]:
#2021
frame = []

df =  pd.read_csv("./2021/202101-citibike-tripdata.csv", delimiter=',')
df = df[['tripduration','starttime','start station name','end station name','usertype',"start station latitude","start station longitude","end station latitude","end station longitude"]]
df = df[(df['start station name'].isin(station_2021))&(df['end station name'].isin(station_2021))]
#df = df.drop_duplicates(subset=["start station latitude","start station longitude","end station latitude","end station longitude"])
df = df.drop(labels=['start station name','end station name'], axis=1)
df = df.dropna(axis=0, how='any', subset=["start station latitude","start station longitude","end station latitude","end station longitude"])
frame.append(df)

months = ['02','03','04','05','06','07','08','09','10','11','12']

for month in months:
    df =  pd.read_csv("./2021/2021"+month+"-citibike-tripdata.csv", delimiter=',')
    df = df[['started_at','ended_at','start_station_name','end_station_name','member_casual']]
    df['started_at'] = pd.to_datetime(df['started_at'], format = '%Y-%m-%d')
    df['ended_at'] = pd.to_datetime(df['ended_at'], format = '%Y-%m-%d')
    df['tripduration'] = (df['ended_at']-df['started_at']).dt.total_seconds()   # .astype(int) # 化为整数？
    df = df.drop(labels=['ended_at'],axis=1)
    df['member_casual'].loc[df['member_casual'] == 'member'] = 'Subscriber'
    df['member_casual'].loc[df['member_casual'] == 'casual'] = 'Customer'
    df.rename(columns={"member_casual":"usertype", 'started_at':'starttime','start_station_name':'start station name','end_station_name':'end station name'},inplace=True )
    df = df[(df['start station name'].isin(station_2021))&(df['end station name'].isin(station_2021))]
    #df = df.drop_duplicates(subset=["start station latitude","start station longitude","end station latitude","end station longitude"])
    df = df.drop(labels=['start station name','end station name'], axis=1)
    df = df.dropna(axis=0, how='any', subset=["start station latitude","start station longitude","end station latitude","end station longitude"])
    frame.append(df)
result_2021 = pd.concat(frame)

## Calculate Euclidean Distance

In [ ]:
pip install geopy

In [ ]:
from geopy.distance import great_circle
def get_straight(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):
    start_point = (pickup_lon,pickup_lat)
    end_ponit = (dropoff_lon,dropoff_lat)
    straight = great_circle(start_point, end_ponit).meters

    return straight

In [ ]:
straight = []
for index,row in result_2019.iterrows():
    straight.append(get_straight(row['start station longitude'], row['start station latitude'], row['end station longitude'], row['end station latitude']))
result_2019['straight'] = straight
result_2019_ = result_2019.drop(labels=["start station latitude","start station longitude","end station latitude","end station longitude"], axis=1)
result_2019_.to_csv('E:/SI/data/Station/2019available_station-tripdata.csv',index=False)

In [ ]:
straight = []
for index,row in result_2020.iterrows():
    straight.append(get_straight(row['start station longitude'], row['start station latitude'], row['end station longitude'], row['end station latitude']))
result_2020['straight'] = straight
result_2020_ = result_2020.drop(labels=["start station latitude","start station longitude","end station latitude","end station longitude"], axis=1)
result_2020_.to_csv('E:/SI/data/Station/2020available_station-tripdata.csv',index=False)

In [ ]:
straight = []
for index,row in result_2021.iterrows():
    straight.append(get_straight(row['start station longitude'], row['start station latitude'], row['end station longitude'], row['end station latitude']))
result_2021['straight'] = straight
result_2021_ = result_2021.drop(labels=["start station latitude","start station longitude","end station latitude","end station longitude"], axis=1)
result_2021_.to_csv('E:/SI/data/Station/2021available_station-tripdata.csv',index=False)

In [ ]:
result_2019_duplicates = result_2019.drop_duplicates(subset=['straight'])
result_2019_duplicates = result_2019_duplicates.drop(labels=['tripduration','starttime','usertype'])
result_2020_duplicates = result_2020.drop_duplicates(subset=['straight'])
result_2020_duplicates = result_2020_duplicates.drop(labels=['tripduration','starttime','usertype'])
result_2021_duplicates = result_2021.drop_duplicates(subset=['straight'])
result_2021_duplicates = result_2021_duplicates.drop(labels=['tripduration','starttime','usertype'])

## Calculate Shortest Distance

In [ ]:
import requests
import folium
import polyline

In [ ]:
pip install folium

In [ ]:
pip install polyline

In [ ]:
session = requests.Session()
def get_distance(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat):

    loc = "{},{};{},{}".format(pickup_lon, pickup_lat, dropoff_lon, dropoff_lat)
    url = "http://router.project-osrm.org/route/v1/driving/"
    r = session.get(url + loc) 
    if r.status_code!= 200:
        return 0
  
    res = r.json() 
    distance = res['routes'][0]['distance']

    return distance

In [ ]:
distance = []
for index,row in result_2019_duplicates.iterrows():
    distance.append(get_distance(row['start station longitude'], row['start station latitude'], row['end station longitude'], row['end station latitude']))
result_2019_duplicates['distance'] = distance


In [ ]:
distance = []
for index,row in result_2020_duplicates.iterrows():
    distance.append(get_distance(row['start station longitude'], row['start station latitude'], row['end station longitude'], row['end station latitude']))
result_2020_duplicates['distance'] = distance

In [ ]:
distance = []
for index,row in result_2021_duplicates.iterrows():
    distance.append(get_distance(row['start station longitude'], row['start station latitude'], row['end station longitude'], row['end station latitude']))
result_2021_duplicates['distance'] = distance

In [ ]:
result_2019_duplicates.to_csv('E:/SI/data/2019available_station-tripdata_distance.csv', index=False)
result_2020_duplicates.to_csv('E:/SI/data/2020available_station-tripdata_distance.csv', index=False)
result_2021_duplicates.to_csv('E:/SI/data/2021available_station-tripdata_distance.csv', index=False)